## This script processes the eplusout.eso file, which contains timeseries outputs of household loads, into pandas dataframes and CSVs. 

In [2]:
import pandas as pd, csv, os

In [32]:
# outputDir = 'EPcomparison_pysimple'
# esoFile = os.path.join(outputDir,'eplusout.eso')

outputDir = 'EPcomparison_gui'
esoFile = os.path.join(outputDir,'in_DT92H.eso')

In [33]:
#Read file as text file
with open(esoFile) as f:
    raw = f.read()

In [34]:
#Break up data into headers and data
headerStart,headerEnd = '\n','End of Data Dictionary'
headers = raw[raw.find(headerStart)+len(headerStart):raw.find(headerEnd)-1]

dataStart = 'RUN PERIOD'
data = raw[raw.find(dataStart):]
data = data[data.find('\n')+len('\n'):] #go to end of line

In [35]:
#Divide headers into hourly, timestep, and run period
headerList = headers.split('\n')

hourlyHeaders = [h for h in headerList if '!Hourly' in h]
tsHeaders = [h for h in headerList if '!TimeStep' in h]
runPeriodHeaders = [h for h in headerList if '!RunPeriod' in h]

hourlyHeaderKeys = [h.split(',')[0] for h in hourlyHeaders]
tsHeaderKeys = [h.split(',')[0] for h in tsHeaders]
runPeriodHeaderKeys = [h.split(',')[0] for h in runPeriodHeaders]

print(hourlyHeaderKeys,tsHeaderKeys)

['827', '963', '1027', '1091', '1155', '1219', '2182', '2194', '2206', '2218', '2230', '2242', '2254', '2266', '2278', '2290', '2302', '14'] ['8', '13', '667']


In [36]:
#Set indexes for timeseries data using arbitrary year (this could instead use the '2,...' rows of data)
hourlyDts = pd.date_range(start='1/1/2023 0:00',end='12/31/2023 23:00',freq='H')
tsDts = pd.date_range(start='1/1/2023 0:00',end='12/31/2023 23:50',freq='10min')

In [37]:
#Turn data as text into data as list
dataList = data.split('\n')

In [38]:
#Combine hourly data into df
dfs = list()
for h in hourlyHeaders:
    headerKey,headerName = h.split(',')[0],h[h.find(',')+len(','):]
    headerName = headerName[headerName.find(',')+len(','):headerName.find(' !')]
    keyData = [float(d.split(',')[1]) for d in dataList if d.split(',')[0] == headerKey]
    dfs.append(pd.Series(keyData,index=pd.DatetimeIndex(hourlyDts),name=headerName))
hourlyDf = pd.concat(dfs,axis=1)
    
#Combine sub-hourly data into df and resample to hourly
dfs = list()
for h in tsHeaders:
    headerKey,headerName = h.split(',')[0],h[h.find(',')+len(','):]
    headerName = headerName[headerName.find(',')+len(','):headerName.find(' !')]
    keyData = [float(d.split(',')[1]) for d in dataList if d.split(',')[0] == headerKey]
    dfs.append(pd.Series(keyData,index=pd.DatetimeIndex(tsDts),name=headerName))
tsDf = pd.concat(dfs,axis=1)
tsDf = tsDf.resample('H').sum() #NOTE this does not work for wetbulb temperature, so ignore that field

#Combine hourly and timestep dfs
df = pd.concat([hourlyDf,tsDf],axis=1)
print(df.shape)

(8760, 21)


In [39]:
df

,"GARAGE ZONE,Zone Air Temperature [C]","LIVING ZONE,Zone Air Temperature [C]","RES DS RES AC ASYS RET AIR ZONE,Zone Air Temperature [C]","RES DS RES FUR GAS ASYS RET AIR ZONE,Zone Air Temperature [C]","UNFINISHED ATTIC ZONE,Zone Air Temperature [C]","UNFINISHED BASEMENT ZONE,Zone Air Temperature [C]",UNIT 1:ELECTRICITYHEATING [J],UNIT 1:ELECTRICITYCOOLING [J],UNIT 1:ELECTRICITYINTERIORLIGHTING [J],CENTRAL:ELECTRICITYEXTERIORLIGHTING [J],...,UNIT 1:ELECTRICITYINTERIOREQUIPMENT [J],UNIT 1:ELECTRICITYFANSHEATING [J],UNIT 1:ELECTRICITYFANSCOOLING [J],UNIT 1:ELECTRICITYWATERSYSTEMS [J],UNIT 1:NATURALGASHEATING [J],UNIT 1:NATURALGASWATERSYSTEMS [J],Electricity:Facility [J],"Environment,Site Outdoor Air Wetbulb Temperature [C]",Electricity:Facility [J],NaturalGas:Facility [J]
2023-01-01 00:00:00,16.206049,20.0,23.00000,20.179303,2.259114,21.034616,69117.518571,0.0,1.723077e+05,38241.750073,...,1.662908e+06,650056.567480,0.0,0.0,2.702599e+07,0.000000e+00,2.597386e+06,-0.983732,2.597386e+06,2.711033e+07
2023-01-01 01:00:00,15.705742,20.0,23.00000,20.168950,1.300912,21.046002,76426.833844,0.0,1.033846e+05,22945.050044,...,1.546710e+06,718801.344421,0.0,0.0,2.988404e+07,0.000000e+00,2.471120e+06,-12.276077,2.471120e+06,2.996838e+07
2023-01-01 02:00:00,15.330108,20.0,23.00000,20.164324,0.829643,20.983805,77840.389101,0.0,6.892306e+04,15296.700029,...,1.539306e+06,732095.960572,0.0,0.0,3.043676e+07,0.000000e+00,2.435363e+06,-10.226056,2.435363e+06,3.048496e+07
2023-01-01 03:00:00,15.072737,20.0,23.00000,20.154467,0.568013,20.930140,79229.330052,0.0,6.892306e+04,15296.700029,...,1.497934e+06,745159.076918,0.0,0.0,3.097986e+07,1.941642e+06,2.408444e+06,-8.358363,2.408444e+06,3.296970e+07
2023-01-01 04:00:00,14.862275,20.0,23.00000,20.147730,0.491664,20.884295,79927.224263,0.0,7.662694e+04,17006.488154,...,1.385181e+06,751722.835643,0.0,0.0,3.125275e+07,0.000000e+00,2.312579e+06,-5.247421,2.312579e+06,3.133709e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-31 19:00:00,9.581379,20.0,21.94511,19.790014,2.597354,18.661463,70722.538597,0.0,1.375716e+06,305324.616040,...,2.788118e+06,665151.927245,0.0,0.0,2.765358e+07,0.000000e+00,5.242991e+06,11.695517,5.242991e+06,2.837649e+07
2023-12-31 20:00:00,9.591539,20.0,21.94511,19.777213,2.487910,18.620094,70696.625310,0.0,1.153250e+06,255950.784739,...,2.882595e+06,664908.210421,0.0,0.0,2.764344e+07,4.700344e+06,5.059220e+06,11.693412,5.059220e+06,3.276549e+07
2023-12-31 21:00:00,9.575537,20.0,21.94511,19.778395,2.396979,18.638507,72054.942208,0.0,8.683802e+05,192727.243838,...,2.792323e+06,677683.304762,0.0,0.0,2.817457e+07,1.951087e+06,4.627129e+06,11.692556,4.627129e+06,3.042687e+07
2023-12-31 22:00:00,9.503085,20.0,21.94511,19.786219,2.224420,18.710635,76528.310163,0.0,7.470278e+05,165794.444846,...,2.399934e+06,719755.738460,0.0,0.0,2.992372e+07,2.568325e+06,4.129652e+06,9.961599,4.129652e+06,3.268482e+07


In [40]:
print(df.sum())

GARAGE ZONE,Zone Air Temperature [C]                             1.366352e+05
LIVING ZONE,Zone Air Temperature [C]                             1.821466e+05
RES DS RES AC ASYS RET AIR ZONE,Zone Air Temperature [C]         1.960082e+05
RES DS RES FUR GAS ASYS RET AIR ZONE,Zone Air Temperature [C]    1.773259e+05
UNFINISHED ATTIC ZONE,Zone Air Temperature [C]                   1.083850e+05
UNFINISHED BASEMENT ZONE,Zone Air Temperature [C]                1.844378e+05
UNIT 1:ELECTRICITYHEATING [J]                                    3.304944e+08
UNIT 1:ELECTRICITYCOOLING [J]                                    6.045637e+09
UNIT 1:ELECTRICITYINTERIORLIGHTING [J]                           4.523283e+09
CENTRAL:ELECTRICITYEXTERIORLIGHTING [J]                          1.003892e+09
CENTRAL:ELECTRICITYGARAGELIGHTING [J]                            1.248057e+08
UNIT 1:ELECTRICITYINTERIOREQUIPMENT [J]                          1.364841e+10
UNIT 1:ELECTRICITYFANSHEATING [J]                               

In [22]:
df.to_csv('eplusoutTimeSeriesData.csv')